## Close form vs nomerical rate of change of cost function
- instead of numerically calculating the rate of the cost function using Limits 
- we calculate the derivative of the cost function using the close form solution
  - done mathematically by taking the derivative of the cost function
- this is a more efficient as we don't have to calculate the $L$ twice like we did in the limit 

# **Closed Form Evaluation of the Gradient**
- mathimatically calculating the derivative of the cost function
$$
L = \frac{1}{N} \sum_{i=0}^{N-1}\sqrt{(x_i - x_p)^2 + (y_i - y_p)^2}
$$

$$
\frac{\partial{L}}{\partial{x_p}} = \frac{-1}{N}* \sum_{i =0} ^{N-1} ((X_i - x_p)^2 + (y_i - y_p)^2)^{\frac{-1}{2}} * (x_i - x_p)
$$

$$
\frac{\partial{L}}{\partial{y_p}} = \frac{-1}{N}* \sum_{i =0} ^{N-1} ((X_i - x_p)^2 + (y_i - y_p)^2)^{\frac{-1}{2}} * (y_i - y_p)
$$

- calculating using the close form solution is cheaper computionally
$$
\frac{\partial{L}}{\partial{x}} = \lim_{h \rightarrow 0} \frac{L(x+h, y) - L(x,y)}{h}
$$

$$
\frac{\partial{L}}{\partial{y}} = \lim_{h \rightarrow 0} \frac{L(x, y+h) - L(x,y)}{h}
$$
- Looping over the all data is done twice each iteration in the numerical methode 
  - 1 time for $ L(x+h, y) $
  - 1 time for $ L(x, y)$
- where in the close form, we're iterating over the whole data just 1 time each iteration 





In [1]:
import functions as fn
import timeit
%load_ext autoreload
%autoreload 2


In [2]:
data_x, data_y = fn.gen_pts_(1000)
xp, yp = 5, 5
H = 0.001

def calc_numeric():
    dl_dx = fn.loss(xp + H, yp, data_x, data_y) - fn.loss(xp, yp, data_x, data_y)
    dl_dy = fn.loss(xp, yp+H , data_x, data_y) - fn.loss(xp, yp, data_x, data_y)
    return dl_dx/H, dl_dy/H

def close_form(xp, yp, data_x, data_y):
    sum_x, sum_y = 0, 0
    for xi, yi in zip(data_x, data_y):
        inv_sqrt = ((xi - xp)**2 + (yi - yp)**2) ** (-0.5)
        sum_x += (xi - xp) * inv_sqrt
        sum_y += (yi - yp) * inv_sqrt
    return -sum_x/len(data_x), -sum_y/len(data_y)

In [3]:
calc_numeric(), close_form(xp, yp, data_x, data_y)

((0.706413257168137, 0.7063635749888775),
 (0.7063740168102004, 0.7063243377191615))

In [4]:
## calculate the avg time using timeit 

numerical_time = timeit.timeit(calc_numeric, number=100)
close_form_time = timeit.timeit(lambda: close_form(xp, yp, data_x, data_y), number=100)
print(f"close form is faster than numerical method by {(numerical_time/close_form_time):.2f} times")

close form is faster than numerical method by 2.97 times


In [5]:
import torch 
data = torch.tensor([data_x, data_y], requires_grad=False).T
pnt = torch.tensor([xp, yp], requires_grad=True, dtype=torch.float32)


def calc_torch(data, pnt):
    loss = torch.sqrt(torch.sum((data - pnt)**2, dim=1)).mean()
    loss.backward()
    return pnt.grad
calc_torch(data, pnt)

tensor([0.7064, 0.7063])

In [6]:
torch_time = timeit.timeit(lambda: calc_torch(data, pnt), number=100)
print(f"torch auto grad is faster than close form by {(close_form_time/torch_time):.2f} times")
print(f"torch auto grad is faster than numerical method by {(numerical_time/torch_time):.2f} times")

torch auto grad is faster than close form by 3.71 times
torch auto grad is faster than numerical method by 11.03 times


More about the torch autograd later 